# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [26]:
filepath = 'data/song_data'
song_files = get_files(filepath)
print(song_files[0])
print(len(song_files))

/home/chinanu/anaconda3/envs/udacity_conda_env/project-template/data/song_data/A/B/C/TRABCYE128F934CE1D.json
71


In [6]:
df = pd.read_json(song_files[0], lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AREVWGE1187B9B890A,-13.442,-41.9952,Noci (BA),Bitter End,SOFCHDR12AB01866EF,Living Hell,282.43546,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_records = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data = song_records.values[0].tolist()
song_data

['SOFCHDR12AB01866EF', 'Living Hell', 'AREVWGE1187B9B890A', 0, 282.43546]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_records = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data = artist_records.values[0].tolist()
artist_data

['AREVWGE1187B9B890A', 'Bitter End', 'Noci (BA)', -13.442, -41.9952]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
# cur.execute(artist_table_insert, artist_data)
cur.execute(artist_table_insert, ('AREVWGE1187B9B890A', 'Bitter End', 'Noci (BA)', -13.442, -41.9952))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
filepath = 'data/log_data'
log_files = get_files(filepath)

In [12]:
log_df = pd.read_json(log_files[0], lines=True)
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,689,None,200,1542592468796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
1,Explosions In The Sky,Logged In,Adelyn,F,0,Jordan,497.47546,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.540131e+12,458,Your Hand In Mine,200,1542592496796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",7
2,None,Logged In,Adelyn,F,1,Jordan,NaN,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,Logout,1.540131e+12,458,None,307,1542592497796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",7
3,None,Logged Out,None,None,2,None,NaN,free,None,GET,Home,NaN,458,None,200,1542592500796,None,
4,None,Logged Out,None,None,3,None,NaN,free,None,PUT,Login,NaN,458,None,307,1542592501796,None,


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [13]:
log_df = log_df[log_df.page.eq('NextSong')]
log_df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Explosions In The Sky,Logged In,Adelyn,F,0,Jordan,497.47546,free,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.540131e+12,458,Your Hand In Mine,200,1542592496796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",7
6,Paul Van Dyk Featuring Jessica Sutta,Logged In,Layla,F,0,Griffin,425.66485,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,672,White Lies (Dave Spoon Remix),200,1542592893796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
7,Tim Hughes,Logged In,Layla,F,1,Griffin,323.47383,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,672,God of Justice,200,1542593318796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
8,David Cassidy & The Partridge Family,Logged In,Layla,F,2,Griffin,227.73506,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,672,I'll Meet You Halfway,200,1542593641796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
9,Snow Patrol,Logged In,Layla,F,3,Griffin,200.93342,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,672,Crack The Shutters,200,1542593868796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [14]:
ts_data = log_df['ts']
print(ts_data)
t = pd.to_datetime(ts_data, unit = 'ms')
t.head()

1      1542592496796
6      1542592893796
7      1542593318796
8      1542593641796
9      1542593868796
           ...      
322    1542670421796
323    1542670670796
324    1542670956796
325    1542671236796
326    1542671456796
Name: ts, Length: 276, dtype: int64


1   2018-11-19 01:54:56.796
6   2018-11-19 02:01:33.796
7   2018-11-19 02:08:38.796
8   2018-11-19 02:14:01.796
9   2018-11-19 02:17:48.796
Name: ts, dtype: datetime64[ns]

In [15]:
time_data = (t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [16]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,timestamp,hour,day,week,month,year,weekday
1,2018-11-19 01:54:56.796,1,19,47,11,2018,0
6,2018-11-19 02:01:33.796,2,19,47,11,2018,0
7,2018-11-19 02:08:38.796,2,19,47,11,2018,0
8,2018-11-19 02:14:01.796,2,19,47,11,2018,0
9,2018-11-19 02:17:48.796,2,19,47,11,2018,0


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [17]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [18]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [19]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [20]:
for index, row in log_df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    row_t = pd.to_datetime(row.ts, unit = 'ms')
    songplay_data = ( row_t, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [21]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.